In [14]:
import os
import graphlab as gl

In [15]:
if os.path.exists('wikipedia_w0'):
    docs = gl.SFrame('wikipedia_w0')
else:
    docs = gl.SFrame.read_csv('http://s3.amazonaws.com/dato-datasets/wikipedia/raw/w0.csv', header=False)
    docs.save('wikipedia_w0')

PROGRESS: Downloading http://s3.amazonaws.com/dato-datasets/wikipedia/raw/w0.csv to C:/Users/Nidhin/AppData/Local/Temp/graphlab-Nidhin/8816/000001.csv
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/wikipedia/raw/w0.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.787523 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 12728 lines. Lines per second: 16714.2
PROGRESS: Finished parsing file http://s3.amazonaws.com/dato-datasets/wikipedia/raw/w0.csv
PROGRESS: Parsing completed. Parsed 72270 lines in 1.50901 secs.


In [20]:
docs.head()

X1
anarchism the followingsources cite anarchis ...
alaska alaska waspurchased from the ...
aruba ast utc04 arubawhich has no ...
asteroidsvideogame thegame was conceived by ...
ancientegypt its historyoccurred in a series of ...
alvintoffler a formerassociate editor of ...
algebraicallyclosedfieldas an example the field ...
armedforcesofarmenia thearmed forces of the ...
amberdicelessroleplayinggame the amber diceless ...
anisotropy in the fieldof computer graphics an ...


In [21]:
# Remove stopwords and convert to bag of words
docs = gl.text_analytics.count_words(docs['X1'])
docs = docs.dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)

In [22]:
# Learn topic model
m = gl.topic_model.create(docs)

PROGRESS: Learning a topic model
PROGRESS:        Number of documents     72270
PROGRESS:            Vocabulary size    632779
PROGRESS:    Running collapsed Gibbs sampling
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | 10        | 24.09s        | 4.07199e+006   | 0               |
PROGRESS: +-----------+---------------+----------------+-----------------+


In [23]:
print m.get_topics()


+-------+----------+------------------+
| topic |   word   |      score       |
+-------+----------+------------------+
|   0   |   year   | 0.0158005918447  |
|   0   |  south   | 0.0113969603223  |
|   0   | national | 0.0106480844333  |
|   0   |  years   | 0.00969755496767 |
|   0   |  local   | 0.00809725286648 |
|   1   | company  | 0.0103143586258  |
|   1   |  played  | 0.0102539830853  |
|   1   |  league  | 0.00852525558541 |
|   1   | college  | 0.00839609778369 |
|   1   |   time   | 0.00665132108949 |
+-------+----------+------------------+
[50 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [24]:
print m.get_topics(output_type='topic_words')


+-------------------------------+
|             words             |
+-------------------------------+
| [2009, album, game, music,... |
| [book, house, north, publi... |
| [local, national, south, y... |
| [band, called, including, ... |
| [club, law, school, state,... |
| [city, people, states, tow... |
| [college, company, league,... |
| [age, area, population, st... |
| [de, early, made, team, time] |
| [development, high, season... |
+-------------------------------+
[10 rows x 1 columns]



In [25]:
m

Class                         : TopicModel

Schema
------
Vocabulary Size               : 632779
Topic Size                    : 632779

Settings
--------
Number of Topics              : 10
alpha                         : 5.0
beta                          : 0.1
Iterations                    : 10
Verbose                       : False

Accessible fields             : 
m['topics']                   : An SFrame containing the topics.
m['vocabulary']               : An SArray containing the topics.
Useful methods                : 
m.get_topics()                : Get the most probable words per topic.
m.predict(new_docs)           : Make predictions for new documents.

In [26]:
pred = m.predict(docs)

In [27]:
pred

dtype: int
Rows: 72270
[8L, 6L, 8L, 5L, 5L, 6L, 9L, 4L, 5L, 9L, 6L, 4L, 5L, 2L, 7L, 4L, 8L, 6L, 1L, 5L, 3L, 7L, 2L, 6L, 2L, 1L, 4L, 9L, 0L, 4L, 9L, 9L, 8L, 6L, 4L, 9L, 1L, 9L, 9L, 6L, 0L, 6L, 9L, 4L, 6L, 4L, 9L, 1L, 2L, 1L, 4L, 1L, 5L, 6L, 0L, 9L, 6L, 2L, 3L, 6L, 2L, 5L, 4L, 9L, 6L, 7L, 6L, 9L, 6L, 9L, 0L, 9L, 6L, 6L, 9L, 5L, 1L, 9L, 7L, 3L, 9L, 9L, 9L, 1L, 6L, 6L, 8L, 6L, 6L, 8L, 1L, 5L, 8L, 6L, 7L, 9L, 6L, 5L, 2L, 6L, ... ]

In [28]:
pred = m.predict(docs, output_type='probability')

In [29]:
pred

dtype: array
Rows: 72270
[array('d', [0.05327413984461709, 0.01220865704772475, 0.07140214576396596, 0.0873103958564558, 0.16907140214576397, 0.14021457639659637, 0.09618941916389197, 0.15427302996670367, 0.17351091379948205, 0.042545320014798375]), array('d', [0.10662692043109379, 0.02063746847053428, 0.033937170373767485, 0.19124054116028433, 0.056409080486127035, 0.02247191011235955, 0.22105021783994497, 0.0660398991057097, 0.06649850951616602, 0.21508828250401285]), array('d', [0.17921686746987953, 0.02108433734939759, 0.08333333333333333, 0.08634538152610442, 0.10592369477911646, 0.040160642570281124, 0.09939759036144578, 0.08383534136546185, 0.16967871485943775, 0.13102409638554216]), array('d', [0.04375, 0.08625, 0.145, 0.015, 0.03, 0.21, 0.23, 0.12875, 0.01875, 0.0925]), array('d', [0.02901741044626776, 0.010806483890334201, 0.023814288573143886, 0.12847708625175105, 0.10386231739043426, 0.16409845907544526, 0.17270362217330398, 0.13087852711626977, 0.06904142485491295, 0.16730

In [30]:
m['vocabulary']

dtype: str
Rows: 632779
['definitional', 'diversity', 'countereconomics', 'level', 'simultaneously', 'process', 'technology', 'phenomena', 'attitudes', 'consumer', 'creation', 'selfdetermined', 'ivan', 'popularized', 'se', 'questioned', 'organizing', 'addition', 'germany', 'experiments', 'activity', 'childled', 'series', 'inspiration', 'moderna', 'field', 'country', 'projects', 'pedagogical', 'murray', 'anticlerical', 'fiercely', 'noncoercive', 'rational', 'widely', 'church', 'defiance', 'progressive', 'ferrer', 'freethinker', 'fragments', 'linvention', 'solaire', 'amoureux', 'du', 'onfray', 'joy', 'writing', 'recently', 'proponent', 'consenting', 'camaraderie', 'la', 'emile', 'teufel', 'beginning', 'homosexuality', 'spoke', '18491898', 'reitzel', 'fact', 'boldly', 'campaigned', 'noted', 'frequented', 'villagers', 'discussion', 'margaret', 'lesbian', 'millay', 'edna', 'bisexual', 'sexuality', 'encouraged', 'greenwich', 'lazarus', 'une', 'staunchly', '18721890', 'angela', 'heywood', 'ex